In [1]:
import numpy as np
import pandas as pd

from sklearn.cross_validation import train_test_split
from sklearn import linear_model,tree,ensemble
from sklearn.metrics import accuracy_score,log_loss

import sys
sys.path.insert(0,"../libs")
import data_cleaning as dc

In [2]:
train = pd.read_csv("../data/train.csv")
train = train[train.AnimalType=="Dog"]
train = dc.massage_df(train)

In [3]:
# add dog_size feature
sizes = pd.read_json("../breeds/size.json")
size_dict = sizes.set_index("breed").to_dict()["size"]
train["dog_size"] = train.dog_breed.apply(lambda x:size_dict[x] 
        if x in size_dict.keys() else "other")

In [4]:
mytrain,mytest = train_test_split(train,test_size=0.4)

In [5]:
features = ["neuter_status","age_numeric_years","is_weekend","dog_size"]
# these features are linear combinations of the others
drop_features = ["neuter_status_Unknown","dog_size_other"]

usable_mytrain_idx = mytrain[features].dropna().index
print "train utilization: %1.3f" % (float(len(usable_mytrain_idx))/len(mytrain))
mytrain = mytrain.ix[ usable_mytrain_idx ]
usable_mytest_idx = mytest[features].dropna().index
print "test utilization: %1.3f" % (float(len(usable_mytest_idx))/len(mytest))
mytest = mytest.ix[ usable_mytest_idx ]

mytrain_features = pd.get_dummies(mytrain[features])
mytrain_features = mytrain_features.drop(drop_features,axis=1)
mytrain_outcomes = mytrain.OutcomeType

mytest_features = pd.get_dummies(mytest[features])
mytest_features = mytest_features.drop(drop_features,axis=1)
mytest_outcomes = mytest.OutcomeType

train utilization: 1.000
test utilization: 1.000


In [6]:
# build classifier
#classifier = linear_model.LogisticRegression()
#classifier = tree.DecisionTreeClassifier(max_depth=3)
classifier = ensemble.RandomForestClassifier(max_depth=6)

# train
classifier.fit(mytrain_features,mytrain_outcomes);

In [7]:
predictions = classifier.predict(mytest_features)
prediction_probabilities = classifier.predict_proba(mytest_features)

# check performance
acc = accuracy_score(mytest_outcomes, predictions)
los = log_loss( pd.get_dummies( mytest.OutcomeType ).values, prediction_probabilities)
print("accuracy = {0:1.2f} \nlog-loss = {1:1.3f}".format(acc,los) )

accuracy = 0.55 
log-loss = 1.044
